In [1]:
import pandas as pd

data = pd.read_csv('data/abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [2]:
len(documents)

1103663

In [3]:
print(documents[:5])

                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


In [6]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [7]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/saurabh33.kumar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
print(WordNetLemmatizer().lemmatize('went', pos='v'))

go


In [9]:
stemmer = SnowballStemmer('english')
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data = {'original word': original_words, 'stemmed': singles})

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [10]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [11]:
doc_sample = documents[documents['index'] == 4310].values[0][0]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['rain', 'helps', 'dampen', 'bushfires']


 tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


In [12]:
processed_docs = documents['headline_text'].map(preprocess)

In [13]:
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [14]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [15]:
len(dictionary)

62159

In [16]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [17]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [18]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [20]:
bow_corpus[1]

[(4, 1), (5, 1), (6, 1)]

In [21]:
bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 76 ("bushfir") appears 1 time.
Word 112 ("help") appears 1 time.
Word 484 ("rain") appears 1 time.
Word 4015 ("dampen") appears 1 time.


In [22]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [23]:
corpus_tfidf = tfidf[bow_corpus]

In [24]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5893154583024485),
 (1, 0.3892866165028569),
 (2, 0.49651921997736453),
 (3, 0.5046106271280878)]


In [25]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [26]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.022*"warn" + 0.021*"market" + 0.018*"hospit" + 0.016*"health" + 0.015*"share" + 0.014*"minist" + 0.014*"break" + 0.014*"school" + 0.013*"student" + 0.013*"worker"
Topic: 1 
Words: 0.047*"australian" + 0.025*"adelaid" + 0.020*"test" + 0.017*"tasmanian" + 0.016*"open" + 0.016*"win" + 0.013*"melbourn" + 0.011*"vote" + 0.011*"darwin" + 0.010*"parti"
Topic: 2 
Words: 0.023*"chang" + 0.018*"miss" + 0.017*"tasmania" + 0.014*"farmer" + 0.014*"concern" + 0.013*"fall" + 0.013*"busi" + 0.012*"show" + 0.012*"price" + 0.011*"search"
Topic: 3 
Words: 0.031*"govern" + 0.022*"south" + 0.021*"north" + 0.017*"countri" + 0.016*"rural" + 0.014*"hour" + 0.014*"fund" + 0.014*"west" + 0.014*"indigen" + 0.012*"australia"
Topic: 4 
Words: 0.021*"die" + 0.020*"live" + 0.018*"donald" + 0.017*"coast" + 0.015*"gold" + 0.014*"lose" + 0.013*"park" + 0.012*"royal" + 0.012*"beat" + 0.010*"head"
Topic: 5 
Words: 0.028*"elect" + 0.022*"sydney" + 0.018*"say" + 0.014*"labor" + 0.014*"time" + 0.012*"call

In [27]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

In [28]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.021*"countri" + 0.019*"hour" + 0.010*"podcast" + 0.009*"weather" + 0.008*"christma" + 0.008*"flood" + 0.007*"live" + 0.006*"rain" + 0.006*"drought" + 0.006*"queensland"
Topic: 1 Word: 0.010*"govern" + 0.008*"health" + 0.008*"drum" + 0.007*"fund" + 0.006*"royal" + 0.006*"servic" + 0.005*"school" + 0.005*"commiss" + 0.005*"plan" + 0.005*"job"
Topic: 2 Word: 0.012*"leagu" + 0.009*"final" + 0.007*"rugbi" + 0.006*"octob" + 0.006*"australia" + 0.006*"adelaid" + 0.005*"zealand" + 0.005*"coal" + 0.005*"super" + 0.005*"world"
Topic: 3 Word: 0.008*"abbott" + 0.007*"sexual" + 0.007*"peter" + 0.007*"monday" + 0.007*"say" + 0.007*"violenc" + 0.006*"domest" + 0.006*"toni" + 0.006*"right" + 0.005*"histori"
Topic: 4 Word: 0.018*"charg" + 0.017*"polic" + 0.014*"murder" + 0.013*"woman" + 0.012*"interview" + 0.012*"court" + 0.010*"death" + 0.010*"jail" + 0.009*"alleg" + 0.009*"crash"
Topic: 5 Word: 0.017*"market" + 0.011*"share" + 0.011*"news" + 0.010*"rural" + 0.009*"price" + 0.008*"aus

In [29]:
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [30]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.531390368938446	 
Topic: 0.027*"kill" + 0.024*"home" + 0.019*"crash" + 0.019*"attack" + 0.017*"turnbul" + 0.014*"road" + 0.014*"protest" + 0.014*"driver" + 0.013*"polic" + 0.012*"farm"

Score: 0.3085598647594452	 
Topic: 0.031*"govern" + 0.022*"south" + 0.021*"north" + 0.017*"countri" + 0.016*"rural" + 0.014*"hour" + 0.014*"fund" + 0.014*"west" + 0.014*"indigen" + 0.012*"australia"

Score: 0.020008424296975136	 
Topic: 0.023*"chang" + 0.018*"miss" + 0.017*"tasmania" + 0.014*"farmer" + 0.014*"concern" + 0.013*"fall" + 0.013*"busi" + 0.012*"show" + 0.012*"price" + 0.011*"search"

Score: 0.020007207989692688	 
Topic: 0.022*"warn" + 0.021*"market" + 0.018*"hospit" + 0.016*"health" + 0.015*"share" + 0.014*"minist" + 0.014*"break" + 0.014*"school" + 0.013*"student" + 0.013*"worker"

Score: 0.02000632882118225	 
Topic: 0.021*"die" + 0.020*"live" + 0.018*"donald" + 0.017*"coast" + 0.015*"gold" + 0.014*"lose" + 0.013*"park" + 0.012*"royal" + 0.012*"beat" + 0.010*"head"

Score: 0.02000

In [31]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.6105433702468872	 
Topic: 0.021*"countri" + 0.019*"hour" + 0.010*"podcast" + 0.009*"weather" + 0.008*"christma" + 0.008*"flood" + 0.007*"live" + 0.006*"rain" + 0.006*"drought" + 0.006*"queensland"

Score: 0.22937732934951782	 
Topic: 0.012*"leagu" + 0.009*"final" + 0.007*"rugbi" + 0.006*"octob" + 0.006*"australia" + 0.006*"adelaid" + 0.005*"zealand" + 0.005*"coal" + 0.005*"super" + 0.005*"world"

Score: 0.02001112326979637	 
Topic: 0.010*"govern" + 0.008*"health" + 0.008*"drum" + 0.007*"fund" + 0.006*"royal" + 0.006*"servic" + 0.005*"school" + 0.005*"commiss" + 0.005*"plan" + 0.005*"job"

Score: 0.02001035399734974	 
Topic: 0.018*"charg" + 0.017*"polic" + 0.014*"murder" + 0.013*"woman" + 0.012*"interview" + 0.012*"court" + 0.010*"death" + 0.010*"jail" + 0.009*"alleg" + 0.009*"crash"

Score: 0.02000986970961094	 
Topic: 0.010*"sport" + 0.008*"chang" + 0.007*"climat" + 0.007*"care" + 0.006*"outback" + 0.006*"andrew" + 0.005*"scott" + 0.005*"speak" + 0.005*"smith" + 0.005*"law"


In [32]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.3241119384765625	 Topic: 0.034*"trump" + 0.025*"world" + 0.024*"australia" + 0.016*"final" + 0.016*"china"
Score: 0.2095828801393509	 Topic: 0.022*"warn" + 0.021*"market" + 0.018*"hospit" + 0.016*"health" + 0.015*"share"
Score: 0.18333062529563904	 Topic: 0.038*"polic" + 0.028*"charg" + 0.026*"court" + 0.020*"murder" + 0.017*"face"
Score: 0.1828743815422058	 Topic: 0.028*"elect" + 0.022*"sydney" + 0.018*"say" + 0.014*"labor" + 0.014*"time"
Score: 0.016688944771885872	 Topic: 0.031*"govern" + 0.022*"south" + 0.021*"north" + 0.017*"countri" + 0.016*"rural"
Score: 0.016682250425219536	 Topic: 0.047*"australian" + 0.025*"adelaid" + 0.020*"test" + 0.017*"tasmanian" + 0.016*"open"
Score: 0.016682250425219536	 Topic: 0.023*"chang" + 0.018*"miss" + 0.017*"tasmania" + 0.014*"farmer" + 0.014*"concern"
Score: 0.016682250425219536	 Topic: 0.021*"die" + 0.020*"live" + 0.018*"donald" + 0.017*"coast" + 0.015*"gold"
Score: 0.016682250425219536	 Topic: 0.037*"year" + 0.030*"queensland" + 0.022